<a href="https://colab.research.google.com/github/tomknightatl/usccb-parish-extraction/blob/main/notebooks/01_Build_Dioceses_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build Dioceses Database

This notebook scrapes the USCCB website to build the initial dioceses database.

**What this does**:
- Sets up the complete environment (no separate setup notebook needed)
- Scrapes diocese information from the USCCB website
- Extracts name, address, and website for each diocese
- Saves the data to your Supabase database
- Provides downloadable CSV backup

In [1]:
# Cell 1: Complete Environment Setup
import os
import sys
import warnings
warnings.filterwarnings('ignore')

print("🚀 Setting up USCCB Parish Extraction Environment...\n")

# Step 1: Clone repository if needed
repo_path = '/content/usccb-parish-extraction'
if not os.path.exists(repo_path):
    print("📁 Cloning repository...")
    !git clone https://github.com/tomknightatl/usccb-parish-extraction.git
    print("✅ Repository cloned")
else:
    print("✅ Repository already exists")
    os.chdir(repo_path)
    !git pull --quiet
    print("✅ Repository updated")

# Step 2: Set working directory and Python path
os.chdir(repo_path)
if repo_path not in sys.path:
    sys.path.insert(0, repo_path)
print(f"📂 Working directory: {os.getcwd()}")

# Step 3: Install required packages
print("\n📦 Installing packages...")
!pip install --quiet selenium==4.15.0 webdriver-manager==4.0.1
!pip install --quiet beautifulsoup4==4.12.2 lxml
!pip install --quiet google-generativeai==0.3.0 tenacity==8.2.3
!pip install --quiet "supabase>=2.15.0"
print("✅ Packages installed")

# Step 4: Test imports
print("\n🧪 Testing imports...")
try:
    import requests
    import pandas as pd
    from bs4 import BeautifulSoup
    import time
    from datetime import datetime
    import selenium
    import google.generativeai as genai
    import supabase
    print("✅ External packages imported")

    from config.settings import setup_environment, set_config, get_config
    from src.utils.webdriver import setup_driver, load_page, clean_text
    print("✅ Project modules imported")

except ImportError as e:
    print(f"❌ Import error: {e}")
    print("\n🔧 Try restarting runtime and running this cell again")
    raise

# Step 5: Configure APIs
print("\n🔑 Configuring APIs...")
from google.colab import userdata

try:
    supabase_url = userdata.get('SUPABASE_URL')
    supabase_key = userdata.get('SUPABASE_KEY')
    genai_key = userdata.get('GENAI_API_KEY_USCCB')

    config = setup_environment(
        supabase_url=supabase_url,
        supabase_key=supabase_key,
        genai_api_key=genai_key,
        max_dioceses=10  # Can be changed
    )
    set_config(config)

    print("✅ Configuration complete")
    print(f"   📊 Database: {'Connected' if config.supabase else 'Not connected'}")
    print(f"   🤖 AI: {'Enabled' if config.genai_enabled else 'Mock mode'}")

except Exception as e:
    print(f"❌ Configuration error: {e}")
    print("\n🔧 Make sure to add your API keys to Colab Secrets:")
    print("   • SUPABASE_URL")
    print("   • SUPABASE_KEY")
    print("   • GENAI_API_KEY_USCCB")
    config = None

print("\n🎉 Environment setup complete!")

🚀 Setting up USCCB Parish Extraction Environment...

📁 Cloning repository...
Cloning into 'usccb-parish-extraction'...
remote: Enumerating objects: 225, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 225 (delta 60), reused 13 (delta 13), pack-reused 151 (from 1)
Receiving objects: 100% (225/225), 128.33 KiB | 2.25 MiB/s, done.
Resolving deltas: 100% (112/112), done.
✅ Repository cloned
📂 Working directory: /content/usccb-parish-extraction

📦 Installing packages...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.9/145.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.7/598.7 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 17.

In [2]:
# Cell 2: Fixed Scrape USCCB Dioceses Page (Updated for HTTPS URLs)
import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime

def scrape_dioceses_from_usccb():
    """Scrape dioceses information from USCCB website using requests instead of Selenium"""
    url = "https://www.usccb.org/about/bishops-and-dioceses/all-dioceses"
    print(f"🔍 Scraping dioceses from: {url}")

    try:
        # Use requests instead of Selenium to avoid WebDriver issues
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }

        print("⏳ Loading page (this may take a moment)...")
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')
        print("✅ Page loaded successfully")

        # Find diocese containers
        diocese_containers = soup.find_all('div', class_='views-row')
        print(f"📋 Found {len(diocese_containers)} potential diocese containers")

        dioceses = []

        for i, container in enumerate(diocese_containers):
            diocese_data = extract_diocese_info(container)
            if diocese_data:
                dioceses.append(diocese_data)
                if len(dioceses) % 10 == 0:
                    print(f"   📊 Processed {len(dioceses)} dioceses...")

        print(f"\n✅ Successfully extracted {len(dioceses)} dioceses")
        return dioceses

    except requests.RequestException as e:
        print(f"❌ Error loading page: {e}")
        print("\n🔧 Trying alternative approach...")
        return scrape_dioceses_alternative()
    except Exception as e:
        print(f"❌ Error during scraping: {e}")
        raise

def scrape_dioceses_alternative():
    """Alternative scraping method if main approach fails"""
    print("🔄 Attempting alternative scraping method...")

    # Try different URL or approach
    try:
        # You could implement a fallback method here
        # For now, return empty list but log the attempt
        print("⚠️ Alternative method not yet implemented")
        return []
    except Exception as e:
        print(f"❌ Alternative method also failed: {e}")
        return []

def extract_diocese_info(container):
    """Extract diocese information from a container element"""
    try:
        da_wrap = container.find('div', class_='da-wrap')
        if not da_wrap:
            return None

        # Extract name
        name_div = da_wrap.find('div', class_='da-title')
        if not name_div:
            return None
        name = clean_text(name_div.get_text())

        # Extract address
        address_div = da_wrap.find('div', class_='da-address')
        address_parts = []
        if address_div:
            for div in address_div.find_all('div', recursive=False):
                text = clean_text(div.get_text())
                if text and text.strip():
                    address_parts.append(text)

        address = ", ".join(address_parts) if address_parts else None

        # Extract website with HTTPS conversion
        website_div = da_wrap.find('div', class_='site')
        website = None
        if website_div:
            link = website_div.find('a')
            if link and link.get('href'):
                website = link.get('href')
                # Clean up and ensure HTTPS
                website = normalize_url_to_https(website)

        # Only return if we have a valid name
        if name and len(name.strip()) > 2:
            return {
                'Name': name,
                'Address': address,
                'Website': website,
                'extracted_at': datetime.now().isoformat()
            }

    except Exception as e:
        print(f"⚠️ Error extracting diocese info: {e}")

    return None

def normalize_url_to_https(url):
    """Convert URL to HTTPS format and clean it up"""
    if not url:
        return None

    # Remove any extra whitespace
    url = url.strip()

    # Skip if it's not a valid URL
    if not url or url.lower() in ['none', 'n/a', '']:
        return None

    # Handle URLs that don't start with http/https
    if not url.startswith(('http://', 'https://')):
        url = f"https://{url}"

    # Convert HTTP to HTTPS
    if url.startswith('http://'):
        url = url.replace('http://', 'https://', 1)
        print(f"   🔒 Converted to HTTPS: {url}")

    # Remove trailing slashes for consistency (optional)
    url = url.rstrip('/')

    return url

def clean_text(text):
    """Clean and normalize text"""
    if not text:
        return ""

    # Remove extra whitespace and normalize
    cleaned = ' '.join(text.strip().split())
    return cleaned

# Alternative WebDriver setup that might work better
def setup_driver_fixed():
    """Fixed WebDriver setup with better error handling"""
    try:
        from selenium import webdriver
        from selenium.webdriver.chrome.options import Options
        from selenium.webdriver.chrome.service import Service
        from webdriver_manager.chrome import ChromeDriverManager

        print("🔧 Setting up WebDriver...")

        chrome_options = Options()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        chrome_options.add_argument('--disable-gpu')
        chrome_options.add_argument('--remote-debugging-port=9222')
        chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')

        # Try to install ChromeDriver with specific version handling
        try:
            service = Service(ChromeDriverManager().install())
            driver = webdriver.Chrome(service=service, options=chrome_options)
            print("✅ WebDriver setup successful")
            return driver
        except Exception as driver_error:
            print(f"⚠️ WebDriver setup failed: {driver_error}")
            print("💡 Falling back to requests-based scraping...")
            return None

    except ImportError:
        print("⚠️ Selenium not available, using requests instead")
        return None

def scrape_with_selenium_fixed():
    """Use fixed Selenium approach if available"""
    driver = setup_driver_fixed()
    if not driver:
        print("🔄 WebDriver not available, using requests method...")
        return scrape_dioceses_from_usccb()

    try:
        url = "https://www.usccb.org/about/bishops-and-dioceses/all-dioceses"
        print(f"🔍 Using Selenium to scrape: {url}")

        driver.get(url)
        time.sleep(3)  # Wait for page to load

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Find diocese containers
        diocese_containers = soup.find_all('div', class_='views-row')
        print(f"📋 Found {len(diocese_containers)} potential diocese containers")

        dioceses = []
        for container in diocese_containers:
            diocese_data = extract_diocese_info(container)
            if diocese_data:
                dioceses.append(diocese_data)

        return dioceses

    finally:
        driver.quit()

# Main execution - try multiple approaches
print("🚀 Starting USCCB diocese extraction...\n")

try:
    # First try the requests-based approach
    dioceses_data = scrape_dioceses_from_usccb()

    # If that fails or returns few results, try Selenium
    if not dioceses_data or len(dioceses_data) < 50:
        print("\n🔄 Trying Selenium approach as backup...")
        selenium_data = scrape_with_selenium_fixed()
        if selenium_data and len(selenium_data) > len(dioceses_data):
            dioceses_data = selenium_data

    print(f"\n🎉 Extraction complete! Found {len(dioceses_data)} dioceses.")

except Exception as e:
    print(f"❌ All extraction methods failed: {e}")
    dioceses_data = []

🚀 Starting USCCB diocese extraction...

🔍 Scraping dioceses from: https://www.usccb.org/about/bishops-and-dioceses/all-dioceses
⏳ Loading page (this may take a moment)...
✅ Page loaded successfully
📋 Found 196 potential diocese containers
   🔒 Converted to HTTPS: https://www.bhmdiocese.org/
   🔒 Converted to HTTPS: https://www.aoaj.org
   🔒 Converted to HTTPS: https://www.diocesephoenix.org/
   🔒 Converted to HTTPS: https://www.diocesetucson.org/
   🔒 Converted to HTTPS: https://www.dolr.org/
   🔒 Converted to HTTPS: https://www.ourladyofnareg.org/
   📊 Processed 10 dioceses...
   🔒 Converted to HTTPS: https://www.sfarchdiocese.org/
   🔒 Converted to HTTPS: https://www.dioceseofmonterey.org/
   🔒 Converted to HTTPS: https://www.oakdiocese.org/
   🔒 Converted to HTTPS: https://www.rcbo.org/
   🔒 Converted to HTTPS: https://www.sbdiocese.org/
   🔒 Converted to HTTPS: https://www.dsj.org/
   📊 Processed 20 dioceses...
   🔒 Converted to HTTPS: https://www.stocktondiocese.org/
   🔒 Converte

In [3]:
# Cell 3: Analyze and Display Results
if dioceses_data:
    # Create DataFrame
    df = pd.DataFrame(dioceses_data)

    print(f"📊 DIOCESE EXTRACTION ANALYSIS")
    print(f"{'='*50}")
    print(f"Total dioceses extracted: {len(df)}")
    print(f"Columns: {list(df.columns)}")

    # Statistics
    missing_websites = df['Website'].isna().sum()
    missing_addresses = df['Address'].isna().sum()

    print(f"\n📈 Data Quality:")
    print(f"   ✅ Complete records: {len(df)}")
    print(f"   🌐 With websites: {len(df) - missing_websites} ({(len(df) - missing_websites)/len(df)*100:.1f}%)")
    print(f"   📍 With addresses: {len(df) - missing_addresses} ({(len(df) - missing_addresses)/len(df)*100:.1f}%)")
    print(f"   ❌ Missing websites: {missing_websites}")
    print(f"   ❌ Missing addresses: {missing_addresses}")

    # Show sample data
    print(f"\n📋 Sample Data (first 5 dioceses):")
    print("=" * 50)
    for i, row in df.head().iterrows():
        print(f"{i+1}. {row['Name']}")
        if row['Address']:
            print(f"   📍 {row['Address']}")
        if row['Website']:
            print(f"   🌐 {row['Website']}")
        print()

    if len(df) > 5:
        print(f"... and {len(df) - 5} more dioceses")

    # Check for duplicates
    duplicates = df.duplicated(subset=['Name']).sum()
    if duplicates > 0:
        print(f"\n⚠️ Found {duplicates} potential duplicate dioceses")
        print("   These will be handled during database insertion")
    else:
        print(f"\n✅ No duplicate dioceses found")

else:
    print("❌ No dioceses data was extracted")
    print("\n🔧 Troubleshooting:")
    print("   • Check your internet connection")
    print("   • The USCCB website might be temporarily unavailable")
    print("   • Try running the scraping cell again")
    df = pd.DataFrame()

📊 DIOCESE EXTRACTION ANALYSIS
Total dioceses extracted: 196
Columns: ['Name', 'Address', 'Website', 'extracted_at']

📈 Data Quality:
   ✅ Complete records: 196
   🌐 With websites: 196 (100.0%)
   📍 With addresses: 196 (100.0%)
   ❌ Missing websites: 0
   ❌ Missing addresses: 0

📋 Sample Data (first 5 dioceses):
1. Archdiocese of Mobile
   📍 400 Government Street, Mobile , AL 36602, https://mobarch.org/
   🌐 https://mobarch.org

2. Diocese of Birmingham
   📍 2121 3rd Avenue North, P.O. Box 12047, Birmingham , AL 35202-2047, http://www.bhmdiocese.org/
   🌐 https://www.bhmdiocese.org

3. Archdiocese of Anchorage-Juneau
   📍 225 Cordova Street, Anchorage , AK 99501-2409, http://www.aoaj.org
   🌐 https://www.aoaj.org

4. Diocese of Fairbanks
   📍 1316 Peger Road, Fairbanks , AK 99709-5199, https://dioceseoffairbanks.org/
   🌐 https://dioceseoffairbanks.org

5. Holy Protection of Mary Byzantine Catholic Eparchy of Phoenix
   📍 8105 North 16th Street, Phoenix , AZ 85020, https://ephx.org/
   

In [4]:
# Cell 4: Save to Supabase Database
import pandas as pd
from supabase import create_client, Client
import time
from datetime import datetime

def save_dioceses_to_supabase(dioceses_data):
    """Save dioceses data to Supabase database"""

    if not dioceses_data:
        print("❌ No dioceses data to save")
        return False

    # Get credentials from Colab secrets
    try:
        from google.colab import userdata
        supabase_url = userdata.get('SUPABASE_URL')
        supabase_key = userdata.get('SUPABASE_KEY')

        if not supabase_url or not supabase_key:
            print("❌ Supabase credentials not found in Colab secrets")
            print("\n🔧 Please add these to Colab Secrets:")
            print("   • SUPABASE_URL")
            print("   • SUPABASE_KEY")
            return False

    except Exception as e:
        print(f"❌ Error getting credentials: {e}")
        return False

    # Initialize Supabase client
    try:
        supabase: Client = create_client(supabase_url, supabase_key)
        print("✅ Connected to Supabase")
    except Exception as e:
        print(f"❌ Failed to connect to Supabase: {e}")
        return False

    # Create DataFrame for analysis
    df = pd.DataFrame(dioceses_data)
    print(f"📊 Preparing to save {len(df)} dioceses to database")

    # Insert data in batches
    batch_size = 20
    total_inserted = 0
    total_errors = 0

    try:
        for i in range(0, len(dioceses_data), batch_size):
            batch = dioceses_data[i:i + batch_size]
            batch_num = i//batch_size + 1
            total_batches = (len(dioceses_data) + batch_size - 1) // batch_size

            print(f"📤 Inserting batch {batch_num}/{total_batches}: {len(batch)} dioceses...")

            try:
                # Insert batch to Supabase
                response = supabase.table('Dioceses').insert(batch).execute()

                # Check if insertion was successful
                if hasattr(response, 'data') and response.data:
                    inserted_count = len(response.data)
                    total_inserted += inserted_count
                    print(f"   ✅ Successfully inserted {inserted_count} dioceses")
                else:
                    print(f"   ⚠️ Unexpected response format")
                    total_errors += len(batch)

            except Exception as batch_error:
                error_msg = str(batch_error).lower()

                if 'duplicate' in error_msg or 'unique' in error_msg or 'conflict' in error_msg:
                    print(f"   ⚠️ Some dioceses already exist (duplicates skipped)")
                    # Try to handle duplicates by updating instead
                    try:
                        # For duplicates, we'll count them as "successful" since data exists
                        total_inserted += len(batch)
                        print(f"   ✅ Handled {len(batch)} existing records")
                    except:
                        total_errors += len(batch)
                        print(f"   ❌ Could not handle duplicates")

                elif 'does not exist' in error_msg or 'relation' in error_msg:
                    print(f"   ❌ Table 'Dioceses' does not exist!")
                    print(f"   🔧 Please create the table first with this SQL:")
                    print(f'''
CREATE TABLE "Dioceses" (
    id SERIAL PRIMARY KEY,
    "Name" TEXT NOT NULL,
    "Address" TEXT,
    "Website" TEXT,
    extracted_at TIMESTAMP,
    created_at TIMESTAMP DEFAULT NOW()
);''')
                    return False

                else:
                    print(f"   ❌ Error inserting batch: {batch_error}")
                    total_errors += len(batch)

            # Small delay between batches to avoid rate limiting
            if i + batch_size < len(dioceses_data):
                time.sleep(0.5)

        # Final results
        print(f"\n{'='*60}")
        print(f"📊 DATABASE INSERTION RESULTS")
        print(f"{'='*60}")
        print(f"Total dioceses processed: {len(dioceses_data)}")
        print(f"Successfully saved: {total_inserted}")
        print(f"Errors/Failed: {total_errors}")

        if total_inserted > 0:
            success_rate = (total_inserted / len(dioceses_data)) * 100
            print(f"Success rate: {success_rate:.1f}%")
            print(f"\n🎉 Dioceses database updated successfully!")

            # Verify data in database
            try:
                count_result = supabase.table('Dioceses').select('id', count='exact').execute()
                if hasattr(count_result, 'count') and count_result.count is not None:
                    print(f"📊 Total dioceses in database: {count_result.count}")
                else:
                    print(f"✅ Data saved (unable to verify count)")
            except:
                print(f"✅ Data saved (unable to verify count)")

            return True
        else:
            print(f"\n❌ No dioceses were saved to the database")
            return False

    except Exception as e:
        print(f"❌ Database operation failed: {e}")
        print(f"\n🔧 Troubleshooting:")
        print(f"   • Check your Supabase connection")
        print(f"   • Verify the 'Dioceses' table exists")
        print(f"   • Check your API key permissions")
        return False

# Check if we have data to save
if 'dioceses_data' in locals() and dioceses_data:
    print("💾 Saving dioceses to Supabase database...\n")
    success = save_dioceses_to_supabase(dioceses_data)

    if success:
        print(f"\n✅ Database save completed successfully!")
        print(f"🚀 You can now run parish extraction notebooks")
    else:
        print(f"\n⚠️ Database save failed, but your data is still available")
        print(f"💡 You can export to CSV in the next cell as backup")

elif 'df' in locals() and not df.empty:
    # Convert DataFrame back to list of dicts if needed
    dioceses_data = df.to_dict('records')
    print("🔄 Converting DataFrame to list for database save...")
    success = save_dioceses_to_supabase(dioceses_data)

else:
    print("❌ No dioceses data found to save")
    print("\n🔧 Make sure Cell 2 (scraping) completed successfully")
    print("   • Re-run Cell 2 if needed")
    print("   • Check the dioceses_data variable exists")

💾 Saving dioceses to Supabase database...

✅ Connected to Supabase
📊 Preparing to save 196 dioceses to database
📤 Inserting batch 1/10: 20 dioceses...
   ✅ Successfully inserted 20 dioceses
📤 Inserting batch 2/10: 20 dioceses...
   ✅ Successfully inserted 20 dioceses
📤 Inserting batch 3/10: 20 dioceses...
   ✅ Successfully inserted 20 dioceses
📤 Inserting batch 4/10: 20 dioceses...
   ✅ Successfully inserted 20 dioceses
📤 Inserting batch 5/10: 20 dioceses...
   ✅ Successfully inserted 20 dioceses
📤 Inserting batch 6/10: 20 dioceses...
   ✅ Successfully inserted 20 dioceses
📤 Inserting batch 7/10: 20 dioceses...
   ✅ Successfully inserted 20 dioceses
📤 Inserting batch 8/10: 20 dioceses...
   ✅ Successfully inserted 20 dioceses
📤 Inserting batch 9/10: 20 dioceses...
   ✅ Successfully inserted 20 dioceses
📤 Inserting batch 10/10: 16 dioceses...
   ✅ Successfully inserted 16 dioceses

📊 DATABASE INSERTION RESULTS
Total dioceses processed: 196
Successfully saved: 196
Errors/Failed: 0
Succes

In [5]:
# Cell 5: Export to CSV (Always useful as backup)
if not df.empty:
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f'usccb_dioceses_extracted_{timestamp}.csv'

    try:
        # Save to CSV
        df.to_csv(filename, index=False)
        print(f"📁 Data exported to: {filename}")
        print(f"📊 Exported {len(df)} dioceses")

        # Show file size
        file_size = os.path.getsize(filename) / 1024  # KB
        print(f"📦 File size: {file_size:.1f} KB")

        # Download file in Colab
        try:
            from google.colab import files
            files.download(filename)
            print(f"⬇️ File downloaded to your computer")
            print(f"\n💡 Tip: Keep this CSV as a backup of your dioceses data")
        except ImportError:
            # Not in Colab environment
            print(f"📁 File saved locally: {filename}")

    except Exception as e:
        print(f"❌ Export failed: {e}")

else:
    print("❌ No data to export")
    print("\n🔧 The scraping may have failed. Try:")
    print("   • Re-running Cell 2 (the scraping cell)")
    print("   • Checking your internet connection")
    print("   • Waiting a moment and trying again")

print("\n🎉 Diocese database build complete!")

📁 Data exported to: usccb_dioceses_extracted_20250530_004321.csv
📊 Exported 196 dioceses
📦 File size: 29.9 KB


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⬇️ File downloaded to your computer

💡 Tip: Keep this CSV as a backup of your dioceses data

🎉 Diocese database build complete!
